# Welcome to the Demo!
Lets import some stuff! 

In [ ]:
import keras
import numpy as np
print keras.__version__ # this should be 1.0.0
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution3D, MaxPooling3D
from keras.optimizers import SGD, RMSprop
from keras.utils import np_utils, generic_utils
from keras.models import Sequential
from keras.layers.advanced_activations import LeakyReLU
#sanity check to make sure everything works

### Define some constants

In [ ]:
flies = ['airplane']
sits = ['bench','chair','sofa','stool','toilet']
water = ['cup','bathtub','flower_pot','toilet','vase','bottle','bowl']
batch_size = 64
box_size = 32

classes = [sits, water,flies]
nb_classes = len(classes)
patch_size = box_size

This is our model definition

In [ ]:
def shallow_model():
	nb_filters = 32
	nb_pool = 2
	nb_conv = [5,3]
    #this starts a new empty model
	model = Sequential() #types are sequential or graph. I used sequential.
    #add a convolutional layer
	model.add(Convolution3D(nb_filters, kernel_dim1=nb_conv[0], kernel_dim2=nb_conv[0], 
		kernel_dim3=nb_conv[0], border_mode='valid',
			input_shape=(1, box_size, box_size, box_size),init='he_normal',
			subsample=(2,2,2)
			))
    #this is an activation layer
	model.add(LeakyReLU(alpha=.1))
    #Dropout helps prevent overfitting
	model.add(Dropout(.2))
    #a second convolutional layer
	model.add(Convolution3D(nb_filters, kernel_dim1=nb_conv[1], kernel_dim2=nb_conv[1], 
		kernel_dim3=nb_conv[1], border_mode='valid',init='he_normal'))

	model.add(LeakyReLU(alpha=.1))
    #Some max pooling, to reduce size and help generalize
	model.add(MaxPooling3D())
	model.add(Dropout(.3))
	
	model.add(Flatten())
    #dense layer
	model.add(Dense(128,init='uniform'))
	model.add(Dropout(.4))
    #a final dense layer
	model.add(Dense(nb_classes,init='uniform'))
    #sigmoid will output 3 probabilities which dont have to add up to 3
	model.add(Activation('sigmoid'))

	return model

In [ ]:
model = shallow_model()

### Lets take a look at our model

In [ ]:
from IPython.display import SVG
from keras.utils.visualize_util import model_to_dot
dot = model_to_dot(model,show_shapes=True)
SVG(dot.create(prog='dot', format='svg'))

Ok lets compile our model    
this might take a while (or it might not)

In [ ]:
import time
sgd = SGD(lr=0.01, decay=1e-7, momentum=0.9, nesterov=True)
optimizer = sgd
start = time.time()
model.compile(loss='binary_crossentropy', optimizer=optimizer)
print 'model compiled in ' + str(time.time() - start) + ' seconds'
weights_file="shallow_weights_bench_039.h5"
model.load_weights(weights_file)

These are the model weights and biases. This lets us know the model has loaded correctly.

In [ ]:
print model.layers[0].get_weights()[0].shape #the weights
print model.layers[0].get_weights()[1].shape #the bias

### testing the model
If we want to test this on some actual data, we need a few other things to help us. this loads input data from a file and returns it piece by piece

In [ ]:
import npytar
cfg = {'batch_size' : 32,
	   'reg' : 0.001,
	   'momentum' : 0.9,
	   'dims' : (32, 32, 32),
	   'n_channels' : 1,
	   'n_classes' : 10,
	   'batches_per_chunk': 2,
	   'max_epochs' : 80,
	   'max_jitter_ij' : 2,
	   'max_jitter_k' : 2,
	   'n_rotations' : 12,
	   'checkpoint_every_nth' : 4000,
	   }

size = 2316
size -= size%64

def data_loader(fname):
    dims = cfg['dims']
    chunk_size = cfg['batch_size']*cfg['batches_per_chunk']
    xc = np.zeros((chunk_size, cfg['n_channels'],)+dims, dtype=np.float32)
    reader = npytar.NpyTarReader(fname)
    yc = []
    for ix, (x, name) in enumerate(reader):
        cix = ix % chunk_size
        xc[cix] = x.astype(np.float32)
        feature = [1,0,0] #theyre all benches, this is their feature
        yc.append(feature)
        if len(yc) == chunk_size:
            #print yc
            yield (2.0*xc - 1.0, np.asarray(yc, dtype=np.float32))
            yc = []
            xc.fill(0)

### Test the model, for real

In [ ]:
test_file = "left_out_bench_left_out.tar"
loader = (data_loader(test_file))
progbar = generic_utils.Progbar(size)
y_true = [] #truth
y_pred = [] #predictions
for X_batch,y_batch in loader:
    pred = model.predict_on_batch(X_batch)
    y_pred.append(pred)
    y_true.append(y_batch)
    progbar.add(X_batch.shape[0])

## How did we do?
We now have `y_pred` and `y_true` in memory.    
Lets compare them and see how they match up

These are predictions of what you can do with a bench, given that the model has never seen a bench before. Ideally, the model should say that a bench has a high sittable probability and a low probability elsewhere. 

In [ ]:
#reshape the data because of shape weirdness.
fixed_true = np.vstack(y_true)
fixed_pred = np.vstack(y_pred).reshape(-1,nb_classes)
print fixed_true.shape, fixed_pred.shape

In [ ]:
class_true = fixed_true.flatten()
class_pred = fixed_pred.flatten()

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
from sklearn.metrics import precision_recall_curve,average_precision_score
plt.style.use('seaborn-darkgrid')
precision, recall, thresholds = precision_recall_curve(class_true,class_pred)
mAP = average_precision_score(class_true,class_pred)
plt.figure(figsize=(8, 4))
plt.plot(recall, precision, label='Bench')
plt.xlim([0.0, 1.05])
plt.ylim([0.0, 1.05])
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall for Bench: AUC={0:0.2f}'.format(mAP))
plt.legend(loc="lower left");

## For fun, lets find where precision * recall is largest

then we can make a confusion matrix for this best case scenario

In [ ]:
ptimesr = precision*recall
maxind = np.argmax(ptimesr)
print maxind, ptimesr[maxind],precision[maxind],recall[maxind],thresholds[maxind]
from sklearn.preprocessing import Binarizer
#binarize the labels at this threshold
binarizer = Binarizer(threshold=thresholds[maxind])
binary_predictions = binarizer.fit_transform(class_pred.reshape(-1,1))

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(class_true,binary_predictions)
plt.figure(figsize=(8, 6))
plt.imshow(cm, interpolation='nearest',cmap = plt.cm.Blues)
plt.title("Best Case Bench Confusion Matrix")
tick_marks = np.arange(2)
for x in xrange(2):
    for y in xrange(2):
        plt.annotate(str(cm[x][y]), xy=(y, x), 
                    horizontalalignment='center',
                    verticalalignment='center',
                     color='y',size=24)
plt.xticks(tick_marks, ["Negative","Positive"], rotation=45)
plt.yticks(tick_marks,["Negative","Positive"])
plt.colorbar()
plt.grid(b=False)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label');